In [21]:
import os
import yaml
import pandas
import psycopg2
import numpy as np
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [25]:
def perform_joins(movies, trailers):
    return movies.join(trailers, ['title'])


def send_to_RDS(conn, mov, table):
    cur = conn.cursor()  
    cur.execute("DELETE FROM " + table)
    conn.commit()
    
    mov = mov.toPandas()
    mov.fillna(value=np.nan, inplace=True)
    
    for i in range(len(mov)):
#         for j in mov.columns:
#             mov[j] = mov[j].map(lambda x: x.encode('utf-8'))
        m = mov.iloc[i].values
        
        cur.execute("INSERT INTO " + table +" (title, genre, rated, timestamp,\
                    url) VALUES ('{}', '{}', '{}', '{}', '{}')".format(*m))
        conn.commit()

In [26]:
spark_session = SparkSession.builder.master("local").appName("Flickbot").config("spark.some.config.option", "some-value").getOrCreate()
credentials = yaml.load(open(os.path.expanduser('~/bot-creds.yaml')))
conn = psycopg2.connect(database=credentials['postgres_bot'].get('dbase'),
                        user=credentials['postgres_bot'].get('user'),
                        host=credentials['postgres_bot'].get('host'),
                        port=credentials["postgres_bot"].get("port"),
                        password=credentials['postgres_bot'].get('pass'))

In [27]:
up = spark_session.read.json("s3a://flickbot-api/upcoming.json")
now = spark_session.read.json("s3a://flickbot-api/now_playing.json")
coming_trailers = spark_session.read.json("s3a://flickbot-api/coming_trailers.json")
now_trailers = spark_session.read.json("s3a://flickbot-api/now_trailers.json")
in_theaters = perform_joins(now, now_trailers)
coming_soon = perform_joins(up, coming_trailers)
send_to_RDS(conn, in_theaters, "in_theaters")
send_to_RDS(conn, coming_soon, "coming_soon")